In [1]:
import datetime as dt
from pathlib import Path
import pandas as pd
print("done")

done


In [2]:
import warnings, logging
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("lightgbm").setLevel(logging.ERROR)
logging.getLogger("optuna").setLevel(logging.WARNING)

print("done")

done


In [3]:
from model.features import (
    download_or_load_prices,
    compute_features,
    data_prep_and_feature_engineering,
)
from model.grid_search import run_grid_search
from model.backtest import run_backtest

print("downloaded!")

downloaded!


In [4]:
start = dt.date(2015, 1, 1)
end   = dt.date(2024, 7, 1)

# --- download SPY close prices ---
spy = download_or_load_prices(
    ["SPY"],
    Path("spy_cache.parquet"),
    start,
    end
)["Close"].rename("SPY")

# --- download ^VIX close prices ---
vix = download_or_load_prices(
    ["^VIX"],
    Path("vix_cache.parquet"),
    start,
    end
)["Close"].rename("^VIX")

# --- combine into one DataFrame with plain columns ---
market = pd.concat([spy, vix], axis=1)

print("market columns ->", market.columns.tolist())   # should be ['SPY', '^VIX']



C:\Users\17012\Desktop\market-predictions\model\features.py:27: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download(
C:\Users\17012\Desktop\market-predictions\model\features.py:27: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download(


market columns -> ['SPY', '^VIX']


In [5]:
# 1. raw prices for your ticker
df_prices = download_or_load_prices(
    ["AAPL"],
    Path("price_cache.parquet"),
    start,
    end
)

# 2. compute all features + target
df_feat = compute_features(df_prices, market)

# 3. build inputs for the next helper
feature_list = [c for c in df_feat.columns if c != "Target"]  # list of feature names
tickers      = ["AAPL"]                                       # list of tickers

# 4. prepare training data (function returns X_train_sel, y_train)
X_train, y_train = data_prep_and_feature_engineering(
    tickers,
    feature_list,
    Path("feature_cache.parquet"),
    start,
    end
)

print("X_train shape:", X_train.shape, "| y_train shape:", y_train.shape)


C:\Users\17012\Desktop\market-predictions\model\features.py:27: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download(
C:\Users\17012\Desktop\market-predictions\model\features.py:91: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["VIX_Change"] = vix.pct_change(5)
C:\Users\17012\Desktop\market-predictions\model\features.py:27: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download(


X_train shape: (1860, 23) | y_train shape: (1860,)


In [6]:
# hyper‑parameter search
best_params, best_model = run_grid_search(X_train, y_train)
print("best params:", best_params)

# simple back‑test (use your own test split or walk‑forward logic if you prefer)
results = run_backtest(best_model, X_train, y_train)
results.head()


[I 2025-07-30 11:32:23,379] A new study created in memory with name: no-name-0cb3ff65-ebeb-4313-9398-481cbc189ff1


⏳  Running Optuna grid search (50 trials, CPU/GPU‑safe) …


C:\Users\17012\Desktop\market-predictions\model\grid_search.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https://github.com/optuna/optuna/releases/tag/v3.5.0 for details.
  n_estimators=trial.suggest_int("n", 100, 500, 100),
C:\Users\17012\Desktop\market-predictions\model\grid_search.py:29: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https:

[LightGBM] [Info] Number of positive: 512, number of negative: 976
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5865
[LightGBM] [Info] Number of data points in the train set: 1488, number of used features: 23
[LightGBM] [Info] Using GPU Device: gfx902, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.03 MB) transferred to GPU in 0.006492 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.344086 -> initscore=-0.645138
[LightGBM] [Info] Start training from score -0.645138
Training until validation scores don't improve for 20 rounds


[I 2025-07-30 11:32:28,462] Trial 0 finished with value: 0.0 and parameters: {'n': 100, 'd': 13, 'leaf': 1, 'xgb_n': 250, 'xgb_lr': 0.015552057664651233, 'xgb_subsample': 0.9466490115962272, 'lgb_n': 150, 'lgb_lr': 0.035039058358495793, 'lgb_subsample': 0.8567860913856472}. Best is trial 0 with value: 0.0.


...

C:\Users\17012\Desktop\market-predictions\model\grid_search.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https://github.com/optuna/optuna/releases/tag/v3.5.0 for details.
  n_estimators=trial.suggest_int("n", 100, 500, 100),
C:\Users\17012\Desktop\market-predictions\model\grid_search.py:29: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https:

Training until validation scores don't improve for 20 rounds


[I 2025-07-30 11:33:49,159] Trial 49 finished with value: 0.0 and parameters: {'n': 300, 'd': 11, 'leaf': 4, 'xgb_n': 400, 'xgb_lr': 0.0482631987588909, 'xgb_subsample': 0.9580694608905744, 'lgb_n': 100, 'lgb_lr': 0.016788474547408668, 'lgb_subsample': 0.9098162180152225}. Best is trial 0 with value: 0.0.


Early stopping, best iteration is:
[6]	valid_0's binary_logloss: 0.627506
Trial failed due to The estimator XGBClassifier should be a classifier.; returning 0

===== GridSearch Results (sorted by precision) =====
 value  params_d  params_leaf  params_lgb_lr  params_lgb_n  params_lgb_subsample  params_n  params_xgb_lr  params_xgb_n  params_xgb_subsample
   0.0        13            1       0.035039           150              0.856786       100       0.015552           250              0.946649
   0.0        14            1       0.053676           350              0.662454       100       0.057768           200              0.950961
   0.0        11            2       0.109495           200              0.708018       400       0.274876           250              0.767622
   0.0        14            4       0.061163           350              0.578872       500       0.033161           350              0.999671
   0.0        16            1       0.023290           500              0.883

TypeError: cannot unpack non-iterable NoneType object

In [7]:
print("---------------------------------------")

---------------------------------------


In [14]:
# ── add these two lines FIRST ───────────────────────────────────────────────
import joblib
import warnings; warnings.filterwarnings("ignore", category=UserWarning); warnings.filterwarnings("ignore", category=FutureWarning)
# ────────────────────────────────────────────────────────────────────────────

# 1) run the quiet grid‑search you already have
best_params, best_model = run_grid_search(X_train, y_train)
print("✅ best params:", best_params)

# 2) fit the RandomForest with only the RF‑compatible params
rf_params = {k: v for k, v in best_params.items()
             if k in {"n_estimators", "max_depth", "max_features",
                      "min_samples_split", "min_samples_leaf"}}

best_model = RandomForestClassifier(**rf_params,
                                   n_jobs=-1,
                                   random_state=42).fit(X_train, y_train)
print("🔥  model fitted")

# 3) build bt_data exactly as before  (df_feat, idx_dates, proba …)
# … your existing code that constructs `bt_data` …

# 4) save artefact the way backtest.py expects
art_path = Path("best_model.joblib")
joblib.dump({"model": best_model,
             "features": X_train.columns.tolist(),
             "bt_data": bt_data},
            art_path)
print("📦 artefact saved to", art_path)

# 5) run the back‑test (only a Path arg now)
results = run_backtest(art_path)
results.head()


[I 2025-07-30 11:56:16,260] A new study created in memory with name: no-name-6b297d80-5c41-4e48-875c-5f67079651d2


⏳  Running Optuna grid search (50 trials, CPU/GPU‑safe) …
[LightGBM] [Info] Number of positive: 512, number of negative: 976
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5865
[LightGBM] [Info] Number of data points in the train set: 1488, number of used features: 23
[LightGBM] [Info] Using GPU Device: gfx902, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.03 MB) transferred to GPU in 0.004716 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.344086 -> initscore=-0.645138
[LightGBM] [Info] Start training from score -0.645138
Training until validation scores don't improve for 20 rounds


[I 2025-07-30 11:56:16,682] Trial 0 finished with value: 0.0 and parameters: {'n': 400, 'd': 19, 'leaf': 3, 'xgb_n': 450, 'xgb_lr': 0.08674838178422838, 'xgb_subsample': 0.7056704450279433, 'lgb_n': 300, 'lgb_lr': 0.0863393630435047, 'lgb_subsample': 0.6328920505875433}. Best is trial 0 with value: 0.0.


Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.626751
Trial failed due to The estimator XGBClassifier should be a classifier.; returning 0
[LightGBM] [Info] Number of positive: 512, number of negative: 976
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5865
[LightGBM] [Info] Number of data points in the train set: 1488, number of used features: 23
[LightGBM] [Info] Using GPU Device: gfx902, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.03 MB) transferred to GPU in 0.003479 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.344086 -> initscore=-0.645138
[LightGBM] [Info] Start training from score -0.645138
Training until validation scores don't improve for 20 rounds


[I 2025-07-30 11:56:17,106] Trial 1 finished with value: 0.0 and parameters: {'n': 300, 'd': 13, 'leaf': 4, 'xgb_n': 350, 'xgb_lr': 0.011014719334948854, 'xgb_subsample': 0.553222574462068, 'lgb_n': 400, 'lgb_lr': 0.020903032512830804, 'lgb_subsample': 0.5911701254330661}. Best is trial 0 with value: 0.0.


...

[I 2025-07-30 11:56:43,018] Trial 48 finished with value: 0.0 and parameters: {'n': 200, 'd': 19, 'leaf': 2, 'xgb_n': 500, 'xgb_lr': 0.056473588971029186, 'xgb_subsample': 0.6882781596030829, 'lgb_n': 250, 'lgb_lr': 0.092762076593503, 'lgb_subsample': 0.7721328631080803}. Best is trial 0 with value: 0.0.


Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.626631
Trial failed due to The estimator XGBClassifier should be a classifier.; returning 0
[LightGBM] [Info] Number of positive: 512, number of negative: 976
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5865
[LightGBM] [Info] Number of data points in the train set: 1488, number of used features: 23
[LightGBM] [Info] Using GPU Device: gfx902, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.03 MB) transferred to GPU in 0.004288 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.344086 -> initscore=-0.645138
[LightGBM] [Info] Start training from score -0.645138
Training until validation scores don't improve for 20 rounds


[I 2025-07-30 11:56:43,653] Trial 49 finished with value: 0.0 and parameters: {'n': 300, 'd': 15, 'leaf': 4, 'xgb_n': 450, 'xgb_lr': 0.010708772110867206, 'xgb_subsample': 0.6119949529991547, 'lgb_n': 150, 'lgb_lr': 0.16387973078776785, 'lgb_subsample': 0.7178529404801477}. Best is trial 0 with value: 0.0.


Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.626303
Trial failed due to The estimator XGBClassifier should be a classifier.; returning 0

===== GridSearch Results (sorted by precision) =====
 value  params_d  params_leaf  params_lgb_lr  params_lgb_n  params_lgb_subsample  params_n  params_xgb_lr  params_xgb_n  params_xgb_subsample
   0.0        19            3       0.086339           300              0.632892       400       0.086748           450              0.705670
   0.0        14            3       0.027473           100              0.529248       300       0.215192           300              0.874546
   0.0        17            1       0.010344           150              0.803746       300       0.148693           500              0.830172
   0.0        14            3       0.052877           450              0.584210       500       0.079553           350              0.741442
   0.0        17            4       0.027250           350              0.914

TypeError: cannot unpack non-iterable NoneType object